This script takes bounding boxes in the yolo-format and generates masks in the mvtec-ad format. Masks that contain **pedestrians** are considered anomalous.

Yolo: [x_center, y_center, width, height] between [0,1]
mvtec-ad: Black png image with white pixels where ground truth is

In [ ]:
%load_ext autoreload
%autoreload 2

import sys
import os.path

sys.path.append("..")

from config import SELECTED_DATASET
from utils.data_loader import *

import fiftyone as fo

In [ ]:
dataset_info = load_dataset_info(
    SELECTED_DATASET,
    config_path="/home/dbogdoll/mcity_data_engine/datasets/datasets.yaml",
)
loader_function = dataset_info.get("loader_fct")
dataset = globals()[loader_function](dataset_info)
dataset.persistent = False

In [ ]:
from PIL import Image, ImageDraw
import plotly.graph_objects as go

target_folder = "/home/dbogdoll/mcity_data_engine/datasets/midadvrb_ano_ped_masks"

anomalous_view = dataset.match_tags("val")
for sample in anomalous_view:
    img_width = sample.metadata.width
    img_height = sample.metadata.height
    filename = os.path.basename(sample.filepath).replace(".jpg", "")
    mask = Image.new("L", (img_width, img_height), 0)  # Create a black image
    draw = ImageDraw.Draw(mask)
    for bbox in sample.ground_truth.detections:
        if bbox.label == "pedestrian":
            # Convert YOLO format in [0,1] to image format
            # Yolo format should be <x-center> <y-center> <width> <height> but is x_min, y_min
            # Bug report: https://github.com/voxel51/fiftyone/issues/4887

            x_min_rel, y_min_rel, width_rel, height_rel = bbox.bounding_box
            x_min = int(x_min_rel * img_width)
            y_min = int(y_min_rel * img_height)
            x_max = int((x_min_rel + width_rel) * img_width)
            y_max = int((y_min_rel + height_rel) * img_height)

            # draw.rectangle([x0, y0, x1, y1], fill=255)  # [x0, y0, x1, y1]
            draw.rectangle([x_min, y_min, x_max, y_max], fill=255)  # [x0, y0, x1, y1]

    # Save the mask
    mask.save(os.path.join(target_folder, f"{filename}.png"))